In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
from typing import List, Dict
import copy

import ipywidgets
import numpy as np
import open3d as o3d
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

import casino
from DITTO.data import Hands23Dataset, get_all_runs
from DITTO.config import  TIME_STEPS, RESULTS_DIR
from DITTO.geometry import extract_relative_transforms
from casino.o3d import LineMesh

from PIL import Image
# Activate the interactive stuff
# https://github.com/microsoft/vscode-jupyter/wiki/Using-%25matplotlib-widget-instead-of-%25matplotlib-notebook,tk,etc
import ipympl
%matplotlib widget
# %matplotlib inline

In [ ]:
# Notebook settings
PLOT_2D_TRAJECTORIES = False
PLOT_3D_TRAJECTORIES = True

session_id = "cleanup_box/000"
all_runs = get_all_runs(only_keywords=[str(session_id)])

loaders: List[Hands23Dataset] = []
for ep in tqdm(all_runs, desc="Loading"):
    loaders.append(Hands23Dataset(session_id / ep, lazy_loading=True))

# loaders: List[Hands23Dataset] = [
#    Hands23Dataset(session_id / "000"),
#    Hands23Dataset(session_id / "001")
# Hands23Dataset(session_id / "004") # Demo --> Live pose estimation is not optimal
# Hands23Dataset(session_id / "001") # Demo --> Live pose estimation is not optimal
# ]

print(f"Total runs {len(loaders)}")

In [ ]:
from DITTO.trajectory import Trajectory

num_frames = TIME_STEPS  # number of frames through which we compute flow
trajectories: Dict[int, Trajectory] = {}
for demonstration_index in tqdm(range(len(loaders))):
    trajectories[demonstration_index] = Trajectory.from_hands23(
        loaders[demonstration_index], n_frames=num_frames
    )

# We could pre compute trajectories with .trajectory_2D and .trajectory_3D

In [ ]:
demo_index = 0
trajectory = trajectories[demo_index]

In [ ]:
# Get initial object point cloud
object_pc_start, object_pc_start_rgb = casino.pointcloud.get_pc(
    trajectory.depth_start,
    trajectory.intrinsics,
    mask=trajectory.object_mask_start,
    rgb=trajectory.rgb_start,
)
o3d_object_pc = casino.pointcloud.to_o3d(object_pc_start, object_pc_start_rgb / 255.0)

In [ ]:
# o3d.visualization.draw_geometries([o3d_object_pc])

In [ ]:
world_trajectory_3D = trajectory.trajectory_3D
relative_trajectory_3D = extract_relative_transforms(world_trajectory_3D)

In [ ]:
len(relative_trajectory_3D)

In [ ]:
# Plot relative demo trajectory
o3d_objects = []
print(loaders[demo_index].recording_path)


for idx, transform in enumerate(world_trajectory_3D):
    o3d_objects.append(
        casino.visualization.get_o3d_coordinate_frame(scale=0.1, transform=transform)
    )

    o3d_object_pc_current = copy.deepcopy(o3d_object_pc)
    o3d_object_pc_current.transform(transform @ np.linalg.pinv(world_trajectory_3D[0]))

    o3d_objects.append(o3d_object_pc_current)


center_points = [T[:3, 3] for T in world_trajectory_3D]
lines = np.array(
    [
        np.arange(start=0, stop=len(relative_trajectory_3D)),
        np.arange(start=1, stop=1 + len(relative_trajectory_3D)),
    ],
    dtype=int,
).T

line_mesh = LineMesh(
    points=center_points,
    lines=lines,
    colors=np.array((190, 3, 252)) / 255.0,
    radius=0.005,
)

pcd = loaders[demo_index].get_pointcloud_o3d(0)

all_geometries = [
    pcd,
    # *line_mesh.cylinder_segments,
    *o3d_objects
]


# o3d.visualization.draw_geometries(all_geometries)

In [ ]:
if not "renderer" in locals():
    renderer = casino.o3d.offline_rendering.get_o3d_render(640, 480)

In [ ]:
center = [0, 0, 0.5]  # This is where we look at, 0.5
eye = [0.0, 0.0, -0.5]  # Position in world
up = [
    0.0,
    -1.0,
    0.0,
]
renderer.scene.camera.look_at(center, eye, up)


image = casino.o3d.offline_rendering.render_o3d_mesh(
    all_geometries, renderer=renderer, height_coloring=False
)

episode_id = "_".join(loaders[demo_index].recording_path.parts[-2:])

# out_path = RESULTS_DIR / "demo_renderings" / episode_id

# Single image
# out_path.parent.mkdir(exist_ok=True, parents=True)
# pil_image = Image.fromarray(image)
# pil_image.save(out_path.with_suffix(".png"))

plt.close("all")
plt.imshow(image)
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
fps = 30

# Extracted from MeshLab
rotation_axis = [0.012724, -0.585725, -0.810410]

np_images = casino.o3d.offline_rendering.render_rotate_around_o3d_meshes(
    all_geometries,
    renderer=renderer,
    camera_position=eye,
    rotation_center=center,
    rotation_axis=rotation_axis,
    fps=fps,
    duration=5,
    debug_vis=False,
)

out_path = RESULTS_DIR / "demo_rotation_renderings" / episode_id
out_path.parent.mkdir(exist_ok=True, parents=True)

# casino.images.save_images_to_gif(np_images, out_path.with_suffix(".gif"), fps=fps)
casino.images.save_images_to_mp4(np_images, out_path.with_suffix(".avi"), fps=fps)

# plt.close("all")

# fig, axes = plt.subplots(
#     nrows=1,
#     ncols=len(np_images),
#     squeeze=True,
#     dpi=200,
#     figsize=(4, len(np_images) * 3),
# )

# for idx, image in enumerate(np_images):
#     axes[idx].imshow(image)
#     axes[idx].set_axis_off()

# fig.tight_layout()
# fig.show()